In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import numpy as np
import pandas as pd
from keras.preprocessing.text import Tokenizer
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Conv1D, GlobalMaxPool1D, Input
from keras.layers import Dense, Dropout, MaxPooling1D
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split



data = pd.read_csv('drive/My Drive/ALDAproject/train.csv')

#print(data)
data = data.values
X_train = data[:,1]
Y_train = data[:,2]

Using TensorFlow backend.


In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split

word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)
word_vectorizer.fit(X_train)
X_train = word_vectorizer.transform(X_train)

X_train, X_test, Y_train, Y_test = train_test_split( X_train, Y_train, test_size=0.2, random_state=42)

In [0]:
print(X_train.shape)
print(Y_train)

(127656, 10000)
[1 0 0 ... 1 0 0]


In [0]:
input_seq = Input(shape=(10000, ))
x = Dense(512, activation='relu')(input_seq)
x = Dense(256, activation='relu')(x)
x = Dense(128, activation='relu')(x)
x = Dense(64, activation='relu')(x)
classi = Dense(1, activation='sigmoid')(x)
classifier = Model(input_seq, classi)
classifier.compile(optimizer='adadelta', loss='binary_crossentropy', metrics=['accuracy'])
print(classifier.summary())

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 10000)             0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               5120512   
_________________________________________________________________
dense_2 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_3 (Dense)              (None, 128)               32896     
_________________________________________________________________
dense_4 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 65        
Total params: 5,293,057
Trainable params: 5,293,057
Non-trainable para

In [0]:
classifier.fit(X_train, Y_train, batch_size=512, epochs = 30, 
    verbose=1, validation_data= (X_test, Y_test))

Instructions for updating:
Use tf.cast instead.
Train on 127656 samples, validate on 31915 samples
Epoch 1/30
127656/127656 [==============================] - 20s 160us/step - loss: 0.2241 - acc: 0.9277 - val_loss: 0.1595 - val_acc: 0.9492
Epoch 2/30
127656/127656 [==============================] - 18s 138us/step - loss: 0.1175 - acc: 0.9590 - val_loss: 0.1182 - val_acc: 0.9582
Epoch 3/30
127656/127656 [==============================] - 17s 137us/step - loss: 0.0974 - acc: 0.9650 - val_loss: 0.1141 - val_acc: 0.9609
Epoch 4/30
127656/127656 [==============================] - 17s 135us/step - loss: 0.0873 - acc: 0.9691 - val_loss: 0.1153 - val_acc: 0.9605
Epoch 5/30
127656/127656 [==============================] - 17s 135us/step - loss: 0.0743 - acc: 0.9741 - val_loss: 0.1425 - val_acc: 0.9576
Epoch 6/30
127656/127656 [==============================] - 17s 136us/step - loss: 0.0617 - acc: 0.9797 - val_loss: 0.1292 - val_acc: 0.9602
Epoch 7/30
127656/127656 [=============================

In [0]:
predictions = classifier.predict(X_test)
print(predictions)

for i in range(len(predictions)):
    if predictions[i]>0.5:
        predictions[i] = 1
    else:
        predictions[i] = 0

print(predictions)

[[7.9941750e-04]
 [3.5762787e-07]
 [1.1911988e-04]
 ...
 [5.5176365e-06]
 [9.9955803e-01]
 [5.8994418e-09]]
[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [1.]
 [0.]]


In [0]:
from sklearn.metrics import confusion_matrix, classification_report

print(predictions)
Y_test = [[i] for i in Y_test]

print(Y_test)
print(classification_report(Y_test, predictions))

pd.DataFrame(
   confusion_matrix(Y_test, predictions),
   index = [['Actual', 'Actual'], ['Not Toxic', 'Toxic']],
   columns = [['Predicted', 'Predicted'], ['Not Toxic', 'Toxic']])

[[0.]
 [0.]
 [0.]
 ...
 [0.]
 [1.]
 [0.]]
[[0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [1], [1], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [1], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0], [0

Predicted      
                 Not Toxic Toxic
Actual Not Toxic     28192   667
       Toxic           734  2322

[]
[array([[-0.01969586,  0.01768044, -0.00777268, ...,  0.00625789,
         0.00073308, -0.0033107 ],
       [ 0.00989236,  0.02405954,  0.00145122, ..., -0.02243539,
        -0.00238849,  0.00336044],
       [-0.00429365,  0.01032796, -0.03206109, ..., -0.01289855,
         0.01664544,  0.02343345],
       ...,
       [ 0.00861111, -0.0195963 , -0.00029111, ..., -0.01953948,
        -0.01702588, -0.00781448],
       [-0.00604681, -0.0164614 ,  0.0099281 , ...,  0.01472873,
        -0.01253349, -0.00548438],
       [-0.00357215, -0.00943826, -0.01550163, ..., -0.01676069,
         0.01987262,  0.01082919]], dtype=float32), array([-9.82757099e-03,  2.73111765e-03,  5.66013041e-04, -7.63809821e-03,
        9.45779309e-03, -1.32252667e-02, -2.35932823e-02,  7.43486173e-03,
       -7.14030815e-03,  3.76805197e-03, -1.42102893e-02, -9.87778883e-03,
       -4.75795940e-03, -3.32215568e-03, -1.53876375e-02, -1.10784825e-02,
       -2.15686522e-02, -4.28562006e-03, -7.24051334e-03,  7.222495

In [0]:
data = pd.read_csv('drive/My Drive/ALDAproject/Test_this.csv')

#print(data)
data = data.values
X_train = data[0:8,0]

print(X_train)

['I do not think you are a bad human being your circumstances made you that '
 'He had killed many people in the past and would continue to kill if we do not do somehing about it'
 'You need some help, his retaeded brother was brave enough to serve in the military '
 'The word fuck is very inappropriate please refrain from using. It leaves a bad impression.'
 'The murder rates for those cities are high I would suggest looking after your family. These psychopaths are dagerous, keep yourself safe.'
 'In a time were rape and harassment are widespread, we need to stand up againt the bullies.'
 'I will kill your faimily, nice good wonderful.']


In [0]:
X_test = word_vectorizer.transform(X_train)
predictions = classifier.predict(X_test)
print(predictions)

for i in range(len(predictions)):
    if predictions[i]>0.5:
        predictions[i] = 1
    else:
        predictions[i] = 0

print(predictions)

[[3.2345653e-03]
 [9.9651933e-01]
 [3.4421682e-05]
 [9.9977452e-01]
 [3.1489641e-03]
 [9.1873580e-01]
 [7.6206273e-01]]
[[0.]
 [1.]
 [0.]
 [1.]
 [0.]
 [1.]
 [1.]]
